# Weather Model Training v2.0 - Complete Implementation

Implementasi lengkap mengikuti `training_guide_v2.md`:
- Cyclical Time Features (Sin/Cos)
- Interaction Features (Dew Point)
- Class Balancing (`class_weight='balanced'`)
- Expanding Window Cross-Validation
- Daily Model (agregasi + training)
- Visualisasi Januari 2022 (gap 2 hari)
- 7 File Model Output
- Multi-Step Forecasting

## 1. Persiapan Lingkungan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

try:
    from xgboost import XGBRegressor, XGBClassifier
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False

import joblib
print(f"Libraries loaded | XGBoost: {XGBOOST_AVAILABLE}")

## 2. Load Data

In [ ]:
DATA_PATH = '../data/historical_data_2000_2024.csv'
df = pd.read_csv(DATA_PATH)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)
print(f"Dataset: {len(df):,} rows x {len(df.columns)} cols")
print(f"Range: {df['timestamp'].min()} to {df['timestamp'].max()}")
df.head()

## 3. EDA (Exploratory Data Analysis)

### 3.1 Statistik Deskriptif

In [ ]:
df.describe()

### 3.2 Distribusi Parameter Hourly

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
params = ['temp', 'humidity', 'windspeed', 'sealevelpressure']
titles = ['Temperature (C)', 'Humidity (%)', 'Wind Speed (km/h)', 'Pressure (hPa)']
for ax, param, title in zip(axes.flatten(), params, titles):
    sns.histplot(df[param], kde=True, ax=ax, color='steelblue')
    ax.set_title(f'Distribusi {title}')
plt.suptitle('Distribusi Parameter Cuaca Hourly', fontweight='bold')
plt.tight_layout()
plt.show()

### 3.3 Korelasi Heatmap

In [ ]:
plt.figure(figsize=(12, 8))
cols = ['temp', 'humidity', 'windspeed', 'sealevelpressure', 'rain', 'weather_code']
sns.heatmap(df[cols].corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

### 3.4 Weather Code vs Rain

In [ ]:
df.groupby('weather_code')[['rain']].agg(['mean', 'count'])

## 4. Feature Engineering

### 4.1 Label Encoding

In [ ]:
df_hourly = df.copy()
le_conditions = LabelEncoder()
df_hourly['conditions_encoded'] = le_conditions.fit_transform(df_hourly['conditions'])
le_weather_code = LabelEncoder()
df_hourly['weather_code_encoded'] = le_weather_code.fit_transform(df_hourly['weather_code'])
print("Label encoding done")

### 4.2 [v2.0] Cyclical Time Features

In [ ]:
df_hourly['hour_sin'] = np.sin(2 * np.pi * df_hourly['hour'] / 24)
df_hourly['hour_cos'] = np.cos(2 * np.pi * df_hourly['hour'] / 24)
df_hourly['month_sin'] = np.sin(2 * np.pi * df_hourly['month'] / 12)
df_hourly['month_cos'] = np.cos(2 * np.pi * df_hourly['month'] / 12)
df_hourly['day_of_year'] = df_hourly['timestamp'].dt.dayofyear
df_hourly['doy_sin'] = np.sin(2 * np.pi * df_hourly['day_of_year'] / 365)
df_hourly['doy_cos'] = np.cos(2 * np.pi * df_hourly['day_of_year'] / 365)
print("Cyclical features created")

### 4.3 [v2.0] Interaction Features

In [ ]:
df_hourly['dew_point'] = df_hourly['temp'] - ((100 - df_hourly['humidity']) / 5)
df_hourly['temp_range'] = df_hourly['temp_max_daily'] - df_hourly['temp_min_daily']
df_hourly['humid_temp_ratio'] = df_hourly['humidity'] / (df_hourly['temp'] + 1)
print("Interaction features created")

### 4.4 Lag Features Hourly

In [ ]:
cols = ['temp', 'humidity', 'windspeed', 'sealevelpressure']
for col in cols:
    df_hourly[f'{col}_lag_1'] = df_hourly[col].shift(1)
    df_hourly[f'{col}_lag_24'] = df_hourly[col].shift(24)
    df_hourly[f'{col}_rolling_24'] = df_hourly[col].rolling(24).mean()
    df_hourly[f'{col}_rolling_std_24'] = df_hourly[col].rolling(24).std()

df_hourly = df_hourly.dropna().reset_index(drop=True)
print(f"Hourly: {len(df_hourly):,} rows after dropna")

### 4.5 [v2.0] Preprocessing Data Daily

In [ ]:
# Agregasi hourly ke daily
df_daily = df.groupby(['year', 'month', 'day']).agg({
    'temp': ['min', 'max', 'mean'],
    'humidity': 'mean',
    'windspeed': 'mean',
    'sealevelpressure': 'mean',
    'weather_code': lambda x: x.mode()[0],
    'rain': 'sum'
}).reset_index()

df_daily.columns = ['year', 'month', 'day', 'temp_min', 'temp_max', 'temp_mean',
                    'humidity_avg', 'windspeed_avg', 'pressure_avg',
                    'weather_code_dominant', 'rain_total']

le_weather_code_daily = LabelEncoder()
df_daily['weather_code_dominant_encoded'] = le_weather_code_daily.fit_transform(df_daily['weather_code_dominant'])
print(f"Daily: {len(df_daily):,} rows")

### 4.6 [v2.0] Lag Features Daily dengan Rolling 3d/7d

In [ ]:
# Lag 1 day
for col in ['temp_min', 'temp_max', 'temp_mean', 'humidity_avg', 'windspeed_avg', 'pressure_avg']:
    df_daily[f'{col}_lag_1'] = df_daily[col].shift(1)

# Lag 7 days
df_daily['temp_min_lag_7'] = df_daily['temp_min'].shift(7)
df_daily['temp_max_lag_7'] = df_daily['temp_max'].shift(7)
df_daily['temp_mean_lag_7'] = df_daily['temp_mean'].shift(7)
df_daily['rain_total_lag_1'] = df_daily['rain_total'].shift(1)

# Rolling 3d/7d
df_daily['temp_rolling_3d'] = df_daily['temp_mean'].rolling(3).mean()
df_daily['temp_rolling_7d'] = df_daily['temp_mean'].rolling(7).mean()
df_daily['humidity_rolling_3d'] = df_daily['humidity_avg'].rolling(3).mean()
df_daily['humidity_rolling_7d'] = df_daily['humidity_avg'].rolling(7).mean()

df_daily = df_daily.dropna().reset_index(drop=True)
print(f"Daily after features: {len(df_daily):,} rows")

## 5. Model Training

### 5.1 [v2.0] Expanding Window Cross-Validation Function

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

def evaluate_with_cv(model, X, y):
    """Evaluasi dengan Time Series Cross-Validation"""
    scores = {'r2': [], 'rmse': []}
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        scores['r2'].append(r2_score(y_test, y_pred))
        scores['rmse'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
    return {'R2_mean': np.mean(scores['r2']), 'R2_std': np.std(scores['r2']), 'RMSE_mean': np.mean(scores['rmse'])}

print("CV function ready")

### 5.2 Data Split (HOURLY)

In [ ]:
# Features Hourly
hourly_feature_cols = [
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
    'temp_lag_1', 'temp_lag_24', 'temp_rolling_24',
    'humidity_lag_1', 'humidity_lag_24', 'humidity_rolling_24',
    'windspeed_lag_1', 'windspeed_lag_24', 'windspeed_rolling_24',
    'sealevelpressure_lag_1', 'sealevelpressure_lag_24', 'sealevelpressure_rolling_24',
    'dew_point', 'humid_temp_ratio'
]
hourly_target_reg = ['temp', 'humidity', 'windspeed', 'sealevelpressure']
hourly_target_clf = 'weather_code_encoded'

train_size = int(len(df_hourly) * 0.8)
hourly_train = df_hourly[:train_size]
hourly_test = df_hourly[train_size:]

X_hourly_train = hourly_train[hourly_feature_cols]
X_hourly_test = hourly_test[hourly_feature_cols]
y_hourly_train_reg = hourly_train[hourly_target_reg]
y_hourly_test_reg = hourly_test[hourly_target_reg]
y_hourly_train_clf = hourly_train[hourly_target_clf]
y_hourly_test_clf = hourly_test[hourly_target_clf]

print(f"Hourly Train: {len(X_hourly_train):,} | Test: {len(X_hourly_test):,}")

### 5.3 Data Split (DAILY)

In [ ]:
# Features Daily
daily_feature_cols = [
    'month', 'day',
    'temp_min_lag_1', 'temp_max_lag_1', 'temp_mean_lag_1',
    'humidity_avg_lag_1', 'windspeed_avg_lag_1', 'pressure_avg_lag_1',
    'temp_min_lag_7', 'temp_max_lag_7', 'temp_mean_lag_7',
    'rain_total_lag_1',
    'temp_rolling_3d', 'temp_rolling_7d',
    'humidity_rolling_3d', 'humidity_rolling_7d'
]
daily_target_reg = ['temp_min', 'temp_max', 'temp_mean', 'humidity_avg', 'windspeed_avg', 'pressure_avg']
daily_target_clf = 'weather_code_dominant_encoded'

daily_train_size = int(len(df_daily) * 0.8)
daily_train = df_daily[:daily_train_size]
daily_test = df_daily[daily_train_size:]

X_daily_train = daily_train[daily_feature_cols]
X_daily_test = daily_test[daily_feature_cols]
y_daily_train_reg = daily_train[daily_target_reg]
y_daily_test_reg = daily_test[daily_target_reg]
y_daily_train_clf = daily_train[daily_target_clf]
y_daily_test_clf = daily_test[daily_target_clf]

print(f"Daily Train: {len(X_daily_train):,} | Test: {len(X_daily_test):,}")

### 5.4 Regression Models (HOURLY)

In [ ]:
regression_models = {
    'Linear Regression': LinearRegression(),
    'K-Neighbors': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=10),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
}
if XGBOOST_AVAILABLE:
    regression_models['XGBoost'] = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)

def evaluate_regression(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {'MSE': mse, 'RMSE': np.sqrt(mse), 'MAE': mean_absolute_error(y_true, y_pred), 'R2': r2_score(y_true, y_pred)}

reg_results = []
for name, model in regression_models.items():
    print(f"Training {name}...")
    model.fit(X_hourly_train, y_hourly_train_reg)
    y_pred = model.predict(X_hourly_test)
    metrics = evaluate_regression(y_hourly_test_reg, y_pred)
    metrics['Model'] = name
    reg_results.append(metrics)

df_hourly_reg = pd.DataFrame(reg_results).sort_values('R2', ascending=False)
display(df_hourly_reg[['Model', 'R2', 'RMSE', 'MAE']])
best_hourly_reg_name = df_hourly_reg.iloc[0]['Model']
print(f"Best Hourly Regression: {best_hourly_reg_name}")

### 5.5 [v2.0] Classification with Class Balancing (HOURLY)

In [ ]:
classification_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1),
}
if XGBOOST_AVAILABLE:
    classification_models['XGBoost'] = XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)

clf_results = []
for name, model in classification_models.items():
    print(f"Training {name}...")
    model.fit(X_hourly_train, y_hourly_train_clf)
    y_pred = model.predict(X_hourly_test)
    clf_results.append({'Model': name, 'Accuracy': accuracy_score(y_hourly_test_clf, y_pred), 
                        'F1': f1_score(y_hourly_test_clf, y_pred, average='weighted', zero_division=0)})

df_hourly_clf = pd.DataFrame(clf_results).sort_values('Accuracy', ascending=False)
display(df_hourly_clf)
best_hourly_clf_name = df_hourly_clf.iloc[0]['Model']
print(f"Best Hourly Classification: {best_hourly_clf_name}")

### 5.6 Regression Models (DAILY)

In [ ]:
# Re-initialize models
daily_reg_models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
}
if XGBOOST_AVAILABLE:
    daily_reg_models['XGBoost'] = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)

daily_reg_results = []
for name, model in daily_reg_models.items():
    print(f"Training Daily {name}...")
    model.fit(X_daily_train, y_daily_train_reg)
    y_pred = model.predict(X_daily_test)
    metrics = evaluate_regression(y_daily_test_reg, y_pred)
    metrics['Model'] = name
    daily_reg_results.append(metrics)

df_daily_reg = pd.DataFrame(daily_reg_results).sort_values('R2', ascending=False)
display(df_daily_reg[['Model', 'R2', 'RMSE', 'MAE']])
best_daily_reg_name = df_daily_reg.iloc[0]['Model']
print(f"Best Daily Regression: {best_daily_reg_name}")

### 5.7 Classification (DAILY)

In [ ]:
daily_clf_models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1),
}
if XGBOOST_AVAILABLE:
    daily_clf_models['XGBoost'] = XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)

daily_clf_results = []
for name, model in daily_clf_models.items():
    print(f"Training Daily {name}...")
    model.fit(X_daily_train, y_daily_train_clf)
    y_pred = model.predict(X_daily_test)
    daily_clf_results.append({'Model': name, 'Accuracy': accuracy_score(y_daily_test_clf, y_pred),
                              'F1': f1_score(y_daily_test_clf, y_pred, average='weighted', zero_division=0)})

df_daily_clf = pd.DataFrame(daily_clf_results).sort_values('Accuracy', ascending=False)
display(df_daily_clf)
print(f"Best Daily Classification: {df_daily_clf.iloc[0]['Model']}")

## 6. Analisis Kinerja Individual Parameter

In [ ]:
# Train best models
best_reg_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
best_reg_model.fit(X_hourly_train, y_hourly_train_reg)

best_clf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)
best_clf_model.fit(X_hourly_train, y_hourly_train_clf)

# Per-parameter evaluation
y_pred = best_reg_model.predict(X_hourly_test)
param_results = []
for i, param in enumerate(hourly_target_reg):
    mae = mean_absolute_error(y_hourly_test_reg.iloc[:, i], y_pred[:, i])
    rmse = np.sqrt(mean_squared_error(y_hourly_test_reg.iloc[:, i], y_pred[:, i]))
    r2 = r2_score(y_hourly_test_reg.iloc[:, i], y_pred[:, i])
    param_results.append({'Parameter': param, 'MAE': round(mae, 4), 'RMSE': round(rmse, 4), 'R2': round(r2, 4)})

display(pd.DataFrame(param_results))

### 6.1 Visualisasi Januari 2022 (Gap 2 Hari)

In [ ]:
jan_mask = (df_hourly['year'] == 2022) & (df_hourly['month'] == 1)
df_jan = df_hourly[jan_mask].copy()

if len(df_jan) > 0:
    X_jan = df_jan[hourly_feature_cols]
    y_jan_pred = best_reg_model.predict(X_jan)
    df_jan['temp_pred'] = y_jan_pred[:, 0]
    df_jan['humidity_pred'] = y_jan_pred[:, 1]
    df_jan['windspeed_pred'] = y_jan_pred[:, 2]
    df_jan['pressure_pred'] = y_jan_pred[:, 3]
    
    df_jan_2d = df_jan.set_index('timestamp').resample('2D').mean().reset_index()
    
    fig, axes = plt.subplots(4, 1, figsize=(14, 16))
    params = [('temp', 'temp_pred', 'Temperature (C)', 'red'),
              ('humidity', 'humidity_pred', 'Humidity (%)', 'blue'),
              ('windspeed', 'windspeed_pred', 'Wind Speed (km/h)', 'green'),
              ('sealevelpressure', 'pressure_pred', 'Pressure (hPa)', 'purple')]
    
    for ax, (actual, pred, title, color) in zip(axes, params):
        ax.plot(df_jan_2d['timestamp'], df_jan_2d[actual], '-o', color=color, label='Actual', linewidth=2)
        ax.plot(df_jan_2d['timestamp'], df_jan_2d[pred], '--s', color=color, alpha=0.6, label='Predicted', linewidth=2)
        ax.set_title(f'{title} - January 2022 (2-Day Gap)')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    os.makedirs('outputs', exist_ok=True)
    plt.tight_layout()
    plt.savefig('outputs/jan_2022_predictions_v2.png', dpi=150)
    plt.show()
else:
    print('No data for January 2022')

### 6.2 Confusion Matrix

In [ ]:
if len(df_jan) > 0:
    y_jan_clf_pred = best_clf_model.predict(X_jan)
    cm = confusion_matrix(df_jan['weather_code_encoded'], y_jan_clf_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=le_weather_code.classes_, yticklabels=le_weather_code.classes_)
    plt.title('Confusion Matrix - January 2022')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.savefig('outputs/jan_2022_confusion_v2.png', dpi=150)
    plt.show()
    print(classification_report(df_jan['weather_code_encoded'], y_jan_clf_pred, target_names=[str(c) for c in le_weather_code.classes_]))

## 6.5 Retraining with 100% Data

In [ ]:
# HOURLY MODELS
X_hourly_full = df_hourly[hourly_feature_cols]
y_hourly_reg_full = df_hourly[hourly_target_reg]
y_hourly_clf_full = df_hourly[hourly_target_clf]

print('Training final hourly models...')
hourly_regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
hourly_regressor.fit(X_hourly_full, y_hourly_reg_full)

hourly_classifier = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)
hourly_classifier.fit(X_hourly_full, y_hourly_clf_full)

# DAILY MODELS
X_daily_full = df_daily[daily_feature_cols]
y_daily_reg_full = df_daily[daily_target_reg]
y_daily_clf_full = df_daily[daily_target_clf]

print('Training final daily models...')
daily_regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
daily_regressor.fit(X_daily_full, y_daily_reg_full)

daily_classifier = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)
daily_classifier.fit(X_daily_full, y_daily_clf_full)

print('All 4 final models trained!')

## 7. Save Models (7 Files)

In [ ]:
os.makedirs('models', exist_ok=True)
weather_code_to_rain = {0:0, 1:0, 2:0, 3:0, 51:0.2, 53:0.7, 55:1.1, 61:1.7, 63:4.0, 65:10.3}

# 1. COMBINED MODEL
combined_package = {
    'hourly': {
        'regressor': hourly_regressor,
        'classifier': hourly_classifier,
        'feature_columns': hourly_feature_cols,
        'target_regression': hourly_target_reg,
        'target_classification': 'weather_code',
    },
    'daily': {
        'regressor': daily_regressor,
        'classifier': daily_classifier,
        'feature_columns': daily_feature_cols,
        'target_regression': daily_target_reg,
        'target_classification': 'weather_code_dominant',
    },
    'label_encoder_hourly': le_weather_code,
    'label_encoder_daily': le_weather_code_daily,
    'label_encoder_conditions': le_conditions,
    'weather_code_to_rain': weather_code_to_rain,
    'cyclical_features': ['hour_sin', 'hour_cos', 'month_sin', 'month_cos'],
    'interaction_features': ['dew_point', 'humid_temp_ratio'],
    'version': '2.1',
    'trained_date': datetime.now().isoformat(),
}
joblib.dump(combined_package, 'models/weather_model_combined.pkl')
print('1. Combined model saved')

# 2. HOURLY MODEL
hourly_package = {
    'regressor': hourly_regressor, 'classifier': hourly_classifier,
    'feature_columns': hourly_feature_cols, 'target_regression': hourly_target_reg,
    'label_encoder': le_weather_code, 'weather_code_to_rain': weather_code_to_rain, 'version': '2.1'
}
joblib.dump(hourly_package, 'models/weather_model_hourly.pkl')
print('2. Hourly model saved')

# 3. DAILY MODEL
daily_package = {
    'regressor': daily_regressor, 'classifier': daily_classifier,
    'feature_columns': daily_feature_cols, 'target_regression': daily_target_reg,
    'label_encoder': le_weather_code_daily, 'weather_code_to_rain': weather_code_to_rain, 'version': '2.1'
}
joblib.dump(daily_package, 'models/weather_model_daily.pkl')
print('3. Daily model saved')

In [ ]:
# 4-7. SEPARATE FILES
joblib.dump({'model': hourly_regressor, 'features': hourly_feature_cols, 'target': hourly_target_reg}, 
            'models/weather_model_hourly_regressor.pkl')
print('4. Hourly regressor saved')

joblib.dump({'model': hourly_classifier, 'features': hourly_feature_cols, 'label_encoder': le_weather_code}, 
            'models/weather_model_hourly_classifier.pkl')
print('5. Hourly classifier saved')

joblib.dump({'model': daily_regressor, 'features': daily_feature_cols, 'target': daily_target_reg}, 
            'models/weather_model_daily_regressor.pkl')
print('6. Daily regressor saved')

joblib.dump({'model': daily_classifier, 'features': daily_feature_cols, 'label_encoder': le_weather_code_daily}, 
            'models/weather_model_daily_classifier.pkl')
print('7. Daily classifier saved')

print('\n7 model files saved to models/ folder!')

## 8. Multi-Step Forecasting

In [ ]:
def recursive_forecast(model_reg, model_clf, last_row, feature_cols, n_hours=24):
    predictions = []
    current = last_row.copy()
    for i in range(n_hours):
        X = current[feature_cols].values.reshape(1, -1)
        reg_pred = model_reg.predict(X)[0]
        clf_pred = model_clf.predict(X)[0]
        predictions.append({'hour': i+1, 'temp': reg_pred[0], 'humidity': reg_pred[1],
                           'windspeed': reg_pred[2], 'pressure': reg_pred[3], 'weather_code': clf_pred})
        current['temp_lag_1'] = reg_pred[0]
        current['humidity_lag_1'] = reg_pred[1]
        current['windspeed_lag_1'] = reg_pred[2]
        current['sealevelpressure_lag_1'] = reg_pred[3]
    return pd.DataFrame(predictions)

forecast_72h = recursive_forecast(best_reg_model, best_clf_model, hourly_test.iloc[-1:], hourly_feature_cols, 72)
print('72-hour forecast generated')
forecast_72h.head(10)

## 9. Visualize Forecast

In [ ]:
actual_72h = hourly_test.tail(72).reset_index(drop=True)
start = hourly_test.iloc[-73:-72]
forecast = recursive_forecast(best_reg_model, best_clf_model, start, hourly_feature_cols, 72)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
params = ['temp', 'humidity', 'windspeed', 'pressure']
actual_cols = ['temp', 'humidity', 'windspeed', 'sealevelpressure']

for ax, param, actual_col in zip(axes.flatten(), params, actual_cols):
    ax.plot(range(72), actual_72h[actual_col].values, 'b-', label='Actual', linewidth=2)
    ax.plot(range(72), forecast[param].values, 'r--', label='Forecast', linewidth=2)
    ax.set_title(f'{param} - 72h Recursive Forecast')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/recursive_forecast_72h.png', dpi=150)
plt.show()

## 10. Incremental Learning Impact

In [ ]:
fractions = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
r2_scores = []

for frac in fractions:
    n = int(len(X_hourly_train) * frac)
    temp_model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    temp_model.fit(X_hourly_train.iloc[:n], y_hourly_train_reg.iloc[:n])
    r2 = r2_score(y_hourly_test_reg, temp_model.predict(X_hourly_test))
    r2_scores.append(r2)
    print(f'{int(frac*100):3d}%: R2 = {r2:.4f}')

plt.figure(figsize=(10, 6))
plt.plot([f*100 for f in fractions], r2_scores, 'bo-', linewidth=2, markersize=8)
plt.xlabel('% of Training Data')
plt.ylabel('R2 Score')
plt.title('Impact of Incremental Data on Model Performance')
plt.grid(True, alpha=0.3)
plt.savefig('outputs/incremental_learning.png', dpi=150)
plt.show()

## Summary

### Completed:
1. Cyclical time features (Sin/Cos)
2. Interaction features (Dew Point, Temp Range)
3. Class balancing (`class_weight='balanced'`)
4. Expanding Window CV function
5. Daily model preprocessing + training
6. January 2022 visualization (2-day gap)
7. Multi-step recursive forecasting
8. **7 model files saved**

### Output Files:
```
models/
|- weather_model_combined.pkl      (Hourly + Daily)
|- weather_model_hourly.pkl
|- weather_model_daily.pkl
|- weather_model_hourly_regressor.pkl
|- weather_model_hourly_classifier.pkl
|- weather_model_daily_regressor.pkl
|- weather_model_daily_classifier.pkl
```